# Dostrajanie modeli Open AI

Ten notatnik opiera się na aktualnych wytycznych zawartych w dokumentacji [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) od Open AI.

Dostrajanie poprawia wydajność modeli bazowych dla Twojej aplikacji poprzez ponowne trenowanie ich z dodatkowymi danymi i kontekstem istotnym dla konkretnego przypadku użycia lub scenariusza. Warto zauważyć, że techniki inżynierii promptów, takie jak _few shot learning_ czy _retrieval augmented generation_, pozwalają wzbogacić domyślny prompt o odpowiednie dane, by poprawić jakość. Jednak te podejścia są ograniczone maksymalnym rozmiarem okna tokenów danego modelu bazowego.

Dzięki dostrajaniu faktycznie ponownie trenujemy sam model na wymaganych danych (co pozwala nam użyć znacznie więcej przykładów niż mieści się w maksymalnym oknie tokenów) i wdrażamy _niestandardową_ wersję modelu, która nie wymaga już podawania przykładów podczas wnioskowania. To nie tylko zwiększa skuteczność projektowania promptów (mamy większą swobodę w wykorzystaniu okna tokenów do innych celów), ale potencjalnie także obniża koszty (zmniejszając liczbę tokenów wysyłanych do modelu podczas wnioskowania).

Dostrajanie składa się z 4 kroków:
1. Przygotuj dane treningowe i prześlij je.
1. Uruchom zadanie treningowe, aby uzyskać dostrojony model.
1. Oceń dostrojony model i powtarzaj proces dla lepszej jakości.
1. Wdróż dostrojony model do wnioskowania, gdy będziesz zadowolony z efektów.

Pamiętaj, że nie wszystkie modele bazowe obsługują dostrajanie – [sprawdź dokumentację OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst), aby uzyskać najnowsze informacje. Możesz także dostroić wcześniej dostrojony model. W tym samouczku użyjemy `gpt-35-turbo` jako naszego docelowego modelu bazowego do dostrajania.

---


### Krok 1.1: Przygotuj swój zbiór danych

Zbudujmy chatbota, który pomoże Ci zrozumieć układ okresowy pierwiastków, odpowiadając na pytania o dany pierwiastek w formie limeryku. W _tym_ prostym samouczku stworzymy tylko zbiór danych do trenowania modelu z kilkoma przykładowymi odpowiedziami, które pokazują oczekiwany format danych. W rzeczywistym zastosowaniu musiałbyś przygotować znacznie większy zbiór przykładów. Możesz też skorzystać z otwartego zbioru danych (związanego z Twoją dziedziną), jeśli taki istnieje, i dostosować go do użycia podczas fine-tuningu.

Ponieważ skupiamy się na `gpt-35-turbo` i zależy nam na pojedynczej odpowiedzi (chat completion), możemy przygotować przykłady korzystając z [tego sugerowanego formatu](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst), który odpowiada wymaganiom OpenAI dotyczącym chat completion. Jeśli spodziewasz się rozmów wielotur, powinieneś użyć [formatu dla rozmów wielotur](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst), który zawiera parametr `weight` sygnalizujący, które wiadomości mają być użyte (lub nie) podczas fine-tuningu.

W tym samouczku użyjemy prostszego formatu dla pojedynczej odpowiedzi. Dane zapisane są w [formacie jsonl](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst), gdzie każda linia to jeden rekord w formacie JSON. Poniższy fragment pokazuje 2 przykładowe rekordy – pełny zestaw (10 przykładów), którego użyjemy w tym samouczku, znajdziesz w pliku [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl). **Uwaga:** Każdy rekord _musi_ być zapisany w jednej linii (nie dziel na kilka linii jak w typowym sformatowanym pliku JSON).

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

W rzeczywistym zastosowaniu będziesz potrzebować znacznie większego zbioru przykładów, by uzyskać dobre rezultaty – musisz znaleźć kompromis między jakością odpowiedzi a czasem/kosztem fine-tuningu. My używamy małego zbioru, by szybko przejść przez proces i pokazać, jak to działa. Zobacz [ten przykład z OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) po bardziej zaawansowany samouczek dotyczący fine-tuningu.


### Krok 1.2 Prześlij swój zbiór danych

Prześlij dane za pomocą Files API [zgodnie z opisem tutaj](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Zwróć uwagę, że aby uruchomić ten kod, musisz najpierw wykonać następujące kroki:
 - Zainstalować pakiet `openai` dla Pythona (upewnij się, że używasz wersji >=0.28.0, aby mieć dostęp do najnowszych funkcji)
 - Ustawić zmienną środowiskową `OPENAI_API_KEY` na swój klucz API OpenAI
Aby dowiedzieć się więcej, zobacz [Przewodnik po konfiguracji](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) przygotowany do tego kursu.

Teraz uruchom kod, aby utworzyć plik do przesłania z lokalnego pliku JSONL.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### Krok 2.1: Utwórz zadanie fine-tuningu za pomocą SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### Krok 2.2: Sprawdź status zadania

Oto kilka rzeczy, które możesz zrobić za pomocą API `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` - Wyświetl ostatnie n zadań fine-tuningu
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Pobierz szczegóły konkretnego zadania fine-tuningu
- `client.fine_tuning.jobs.cancel(<job_id>)` - Anuluj zadanie fine-tuningu
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Wyświetl do n zdarzeń z danego zadania
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Pierwszym krokiem w tym procesie jest _weryfikacja pliku treningowego_, aby upewnić się, że dane mają odpowiedni format.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### Krok 2.3: Śledź zdarzenia, aby monitorować postępy


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Krok 2.4: Sprawdź status w panelu OpenAI


Możesz także sprawdzić status, odwiedzając stronę internetową OpenAI i przechodząc do sekcji _Fine-tuning_ na platformie. Tam zobaczysz status aktualnego zadania oraz będziesz mógł śledzić historię wcześniejszych uruchomień. Na tym zrzucie ekranu widać, że poprzednie wykonanie zakończyło się niepowodzeniem, a drugie zakończyło się sukcesem. Dla wyjaśnienia: pierwsze uruchomienie korzystało z pliku JSON z nieprawidłowo sformatowanymi rekordami – po poprawieniu tego problemu, drugie uruchomienie zakończyło się pomyślnie i model został udostępniony do użycia.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.pl.png)


Możesz także zobaczyć komunikaty statusu oraz metryki, przewijając niżej w wizualnym panelu kontrolnym, jak pokazano poniżej:

| Komunikaty | Metryki |
|:---|:---|
| ![Messages](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.pl.png) |  ![Metrics](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.pl.png)|


### Krok 3.1: Pobierz ID i przetestuj dostosowany model w kodzie


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### Krok 3.2: Załaduj i przetestuj dostrojony model w Playground

Teraz możesz przetestować dostrojony model na dwa sposoby. Po pierwsze, możesz wejść do Playground i użyć rozwijanego menu Models, aby wybrać swój nowo dostrojony model z dostępnych opcji. Drugą możliwością jest użycie opcji „Playground” widocznej w panelu Fine-tuning (patrz zrzut ekranu powyżej), która uruchamia _porównawczy_ widok pokazujący wersje modelu bazowego i dostrojonego obok siebie, co pozwala szybko je ocenić.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.pl.png)

Wystarczy uzupełnić kontekst systemowy użyty w danych treningowych i wpisać swoje pytanie testowe. Zauważysz, że po obu stronach pojawi się ten sam kontekst i pytanie. Uruchom porównanie, a zobaczysz różnice w odpowiedziach obu modeli. _Zwróć uwagę, jak dostrojony model generuje odpowiedź w formacie, który podałeś w przykładach, podczas gdy model bazowy po prostu podąża za poleceniem systemowym_.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.pl.png)

Zauważysz też, że porównanie pokazuje liczbę tokenów dla każdego modelu oraz czas potrzebny na wygenerowanie odpowiedzi. **Ten konkretny przykład jest bardzo uproszczony i ma jedynie pokazać proces, a nie odzwierciedla rzeczywistego zbioru danych czy scenariusza**. Możesz zauważyć, że oba przykłady mają taką samą liczbę tokenów (kontekst systemowy i polecenie użytkownika są identyczne), a dostrojony model potrzebuje więcej czasu na wygenerowanie odpowiedzi (model niestandardowy).

W rzeczywistych zastosowaniach nie będziesz używać tak prostych przykładów, lecz dostrajać model na podstawie prawdziwych danych (np. katalog produktów do obsługi klienta), gdzie jakość odpowiedzi będzie dużo bardziej widoczna. W _takim_ przypadku uzyskanie podobnej jakości odpowiedzi od modelu bazowego będzie wymagało bardziej zaawansowanego projektowania promptów, co zwiększy liczbę tokenów i prawdopodobnie wydłuży czas przetwarzania odpowiedzi. _Aby to przetestować, sprawdź przykłady dostrajania w OpenAI Cookbook na początek._



---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony przy użyciu usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby tłumaczenie było poprawne, należy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być traktowany jako źródło nadrzędne. W przypadku informacji o kluczowym znaczeniu zalecane jest skorzystanie z profesjonalnych usług tłumacza. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z korzystania z tego tłumaczenia.
